<a href="https://colab.research.google.com/github/naumovdk/recommender-systems/blob/master/HomeWorks/hw1-matrix-factorization/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных


In [5]:
# !pip install implicit
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [6]:
github_ratings = 'https://raw.githubusercontent.com/naumovdk/recommender-systems/master/HomeWorks/hw1-matrix-factorization/ml-1m/ratings.dat'
ratings = pd.read_csv(github_ratings, delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [7]:
github_movies = 'https://raw.githubusercontent.com/naumovdk/recommender-systems/master/HomeWorks/hw1-matrix-factorization/ml-1m/movies.dat'
movie_info = pd.read_csv(github_movies, delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python', encoding='ISO-8859-1')

Explicit данные

In [8]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [9]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [10]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [11]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [12]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [13]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [14]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [16]:
get_similars(1, model)
model.similar_items(1)

[(1, 0.9034506),
 (3114, 0.72211224),
 (2355, 0.5402882),
 (34, 0.42449737),
 (2384, 0.37631142),
 (588, 0.34754997),
 (2321, 0.33759117),
 (2761, 0.3368096),
 (3887, 0.33242857),
 (1566, 0.3323842)]

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [17]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [18]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [19]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [20]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1182    Aliens (1986)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '2502    Matrix, The (1999)',
 '1884    French Connection, The (1971)',
 '1179    Princess Bride, The (1987)',
 '1892    Rain Man (1988)',
 '3458    Predator (1987)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [21]:
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, make_scorer

По какой-то причине **df.sample()** работает крайне долго, как минимум 95% процентов времени работы алгоритма

Поэтому, чтобы попадать по кешам, я решил идти по датасету подряд

Стало где-то в 30 раз быстрее

In [33]:
class SGD(BaseEstimator):
    def __init__(self, features=40, iterations=10_000, learning_rate=0.01
                 , matrix_reg=0.01, bias_reg=0.01, scale=0.5, verbose=False):
        
        self.features = features
        self.iterations = iterations
        self.learning_rate = learning_rate
        self.scale = scale
        self.scaling = np.power(scale, 1 / iterations)
        
        self.user_reg = matrix_reg
        self.item_reg = matrix_reg
        self.user_bias_reg = bias_reg
        self.item_bias_reg = bias_reg

        self.verbose = verbose


    def fit(self, ratings=ratings):
        self.users = ratings['user_id'].max()
        self.items = ratings['movie_id'].max()
        self.avg = ratings['rating'].mean()

        self.U = np.random.uniform(0, 1 / np.sqrt(self.features), (self.users, self.features))
        self.I = np.random.uniform(0, 1 / np.sqrt(self.features), (self.items, self.features)) 

        self.user_bias = np.zeros((self.users, 1))
        self.item_bias = np.zeros((self.items, 1))
    
        n = len(ratings)
        
        for iter in range(self.iterations):
            i, j, actual = map(int, ratings.loc[iter % n])
            i -= 1
            j -= 1
            
            error = (self.U[i] @ self.I[j].T + self.user_bias[i] + self.item_bias[j].T + self.avg) - actual
            
            self.U[i] -= self.learning_rate * (error * self.I[j] + self.user_reg * self.U[i])
            self.I[j] -= self.learning_rate * (error * self.U[i] + self.item_reg * self.I[j])
            self.user_bias[i] -= self.learning_rate * (error + self.user_bias_reg * self.user_bias[i])
            self.item_bias[j] -= self.learning_rate * (error + self.item_bias_reg * self.item_bias[j])

            self.learning_rate *= self.scaling

            if iter % (self.iterations // 3) == 0:
                rmse = np.linalg.norm(ratings['rating'] - self.predict()[ratings['user_id'] - 1, ratings['movie_id'] - 1])
                rmse /= len(ratings)
                if self.verbose:
                    print(f'rmse {rmse}')
                
            if iter % n == 0:
                ratings = shuffle(ratings)
        
        self.rmse = rmse
        return self


    def predict(self):
        return self.U @ self.I.T + self.user_bias + self.item_bias.T + self.avg

    def similar_items(self, i, n=10):
        i -= 1
        metric = lambda j: np.linalg.norm(self.I[i] - self.I[j])
        distances = [(j + 1, metric(j)) for j in range(len(self.I))]
        return sorted(distances, key=lambda pair: pair[1])[:n]
    
    def recommend(self, i):
        pass

In [23]:
def optimize(model_class):
    param_grid = ParameterGrid({'features': [64]
                               ,'learning_rate': [0.1, 0.05, 0.01]
                               ,'iterations':[2000]
                               ,'verbose':[1]})
    records = []
    for g in param_grid:
        model = model_class()
        model.set_params(**g)
        model.fit()
        records.append(model)
    return min(records, key=lambda m: m.rmse).get_params()

# optimize(SGD)

In [35]:
%%time

sgd = SGD(features=40, iterations=10_000_000, learning_rate=0.01, matrix_reg=0.01, bias_reg=0.01, scale=0.5).fit()
np.save(open('sgd_I.npy', 'wb'), sgd.I)
np.save(open('sgd_U.npy', 'wb'), sgd.U)

Wall time: 26min 5s


Сохраняю матрицы, чтобы можно было редактировать методы класса, не переобучая модельку

In [39]:
sgd = SGD()
sgd.I = np.load('sgd_I.npy')
sgd.U = np.load('sgd_U.npy')

sgd.similar_items(1)
get_similars(1, sgd)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '2728    Big (1988)',
 '584    Aladdin (1992)',
 '2031    Splash (1984)',
 '2618    Tarzan (1999)',
 '2018    Peter Pan (1953)',
 '2225    Antz (1998)',
 '2965    Robin Hood (1973)']

Очень похоже на симилары гугла :)

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

ALS не стоит проверять

In [40]:
class ALS:
    def __init__(self, features=40, iterations=100, learning_rate=0.001, alpha=10, reg=0.1):
        self.features = features
        self.iterations = iterations
        self.reg = reg
        self.alpha = alpha 

    def fit(self, ratings=implicit_ratings):
        self.users = ratings['user_id'].max()
        self.items = ratings['movie_id'].max()
        
        self.p = np.zeros((self.users, self.items))
        self.c = np.zeros((self.users, self.items))
        
        for _, sample in ratings.iterrows():
            user, item, rating = sample
            user -= 1
            item -= 1
            self.p[user, item] = rating
            self.c[user, item] = 1 + rating * self.alpha
        
        self.X = np.random.uniform(0, 1 / np.sqrt(self.features), (self.users, self.features))
        self.Y = np.random.uniform(0, 1 / np.sqrt(self.features), (self.features, self.items)) 

        for iter in range(self.iterations):
            if iter % 2 == 0:
                u = np.random.randint(self.users)
                I = np.eye(self.features)
                Cu = np.diag(self.c[u])
                Y = self.Y
                pu = self.p[u]
                self.X[u] = np.linalg.inv(Y.T @ Y + Y.T @ (Cu - I) @ Y + self.reg * I) @ Y.T @ Cu @ pu
            else:
                i = np.random.randint(self.items)
                I = np.eye(self.features)
                Ci = np.diag(self.c[i]).T
                X = self.X
                pi = self.p[i].T
                self.X[i] = np.linalg.inv(X.T @ X + X.T @ (Ci - I) @ X + self.reg * I) @ X.T @ Ci @ pi

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных